In [16]:
using Distributions
using Optim

In [17]:
function generate_data(N)
    delta1, delta2 = -3 , -2
    beta1, beta2 = 5, 4
    rho = [.5 .5]
    

    Y = zeros(N,2)
    X = zeros(N,2)
    
    for i=1:N
        x1,x2 = rand(Normal(0.0, 1.0),2)
        e1,e2 = rand(Logistic(),2)
        
        odds_lambda = sum(rho .* [x1 x2])
        lambda = exp(odds_lambda)/(1+exp( odds_lambda ))

        select = rand(Categorical([lambda, (1-lambda)]))

        #see logit response i.e. definitely 11
        eq = []
        for y_init=[[0 0],[1 1],[0 1],[1 0]]
            y1,y2 = beta1 * x1 + delta1*y_init[2] + e1 >= 0, beta2 * x2 +delta2*y_init[1] + e2 >= 0

            if (y1==y_init[1]) & (y2==y_init[2])
                append!(eq,[y_init])
            end
        end
        
        y1, y2 = eq[1]
        if size(eq)[1] >= 2
            y1, y2 = eq[select]
        end
        Y[i,:] =[y1 y2]
        X[i,:] =[x1 x2]
    end
    return Y,X
end

Y,X = generate_data(200)

([1.0 0.0; 0.0 1.0; … ; 1.0 0.0; 1.0 0.0], [0.435235 -1.44519; -1.3337 0.513071; … ; 0.89657 -1.08501; 1.05755 0.32544])

In [20]:
function ll_obj(params;xs=0,ys=0)
    n = size(xs)[1]
    k = Integer(size(X)[2]/2) + 1
    p = zeros(2,2,n)

    #solve for the conditional probability firm i does not enter
    #index 1- firm, index2 - firm -i's action
    for a_j=0:1
        util1 =  xs[:,1].*params[1]  .+ a_j.*params[2]
        util2 =  xs[:,2].*params[3]  .+ a_j.*params[4]
        p[1,a_j+1,:] = 1 ./(1 .+ exp.(util1))
        p[2,a_j+1,:] = 1 ./(1 .+ exp.(util2))
    end

    lambda_odds = xs[:,1].*params[5] .+ xs[:,2].*params[6]
    lambda = exp.(lambda_odds ) ./(1. .+ exp.(lambda_odds))
 
    #solve for probablity of nash
    mult_eq = max.(( p[1,2,:]  .- p[1,1,:] ) .* (  p[2,2,:] .- p[2,1,:] ),0)
    prob01 = ( p[1,2,:] ) .* ( 1 .- p[2,1,:] ) - (1 .-lambda).*mult_eq
    prob10 = ( 1 .- p[1,1,:] ) .* ( p[2,2,:] ) - (lambda).*mult_eq
    prob00 = p[1,1,:] .* p[2,1,:]
    prob11 = ( 1 .- p[1,2,:] ) .* ( 1 .- p[2,2,:] )
    
    #compute empirical likelihood
    p00 = (1 .- ys[:,1]).*(1 .- ys[:,2])
    p11 = ys[:,1].*ys[:,2]
    p10 = ys[:,1].* (1 .- ys[:,2])
    p01 = (1 .- ys[:,1]) .* ys[:,2]
    
    ll = p00 .* prob00 + p11 .*prob11 .+ p01 .* prob01 .+ p10 .* prob10

    return -sum(log.(max.(ll,0)))    
end

Y,X = generate_data(1000)
ll_obj([5 -3 4 -2 .5 .5];xs=X,ys=Y)

486.2653001427401

In [21]:
function maximum_likelihood(params;xs=0,ys=0)
    ll_trial(params) = ll_obj(params;xs=xs,ys=ys)
    return optimize(ll_trial, params, LBFGS())
end

result =  maximum_likelihood([5. -3. 4. -2. .5 .5];xs=X,ys=Y)
println(result.minimizer)
println(result.minimum)

[5.40914 -3.28762 4.17223 -2.17112 3.63776 8.9162]
482.31969922781485


In [5]:
function monte_carlo()
    trials = 400
    params_200 = zeros((7,trials))
    params_1000 = zeros((7,trials))
    for i =1:trials
        Y_200, X_200 = generate_data(200)
        trial_200 = maximum_likelihood([5. -3. 4. -2. -2. -2.];xs=X_200,ys=Y_200)
        params_200[:,i] = trial_200.minimizer

        
        Y_1000, X_1000 = generate_data(1000)
        trial_1000 = maximum_likelihood([5. -3. 4. -2. -2. -2.];xs=X_1000,ys=Y_1000)
        params_1000[:,i] = trial_1000.minimizer
    end
    return params_200,params_1000
end

results_200,results_1000 = monte_carlo()

UndefVarError: UndefVarError: Normal not defined

In [6]:
function print_results(params_200, params_1000)
       
    params_200[7,:] = exp.(params_200[7,:])./(1 .+ exp.(params_200[7,:]))
    params_1000[7,:] = exp.(params_1000[7,:])./(1 .+ exp.(params_1000[7,:]))
    
    println("\\begin{center}")
    println("\\begin{table}")
    println("\\normalsize")
    println("\\begin{tabular}{c|c|c|c|c|c}")
    print("& True & Mean & SD  &  Mean  & SD  \\\\\n")
    print("& &  (200 obs) &  (200 obs) &  (1000 obs) &  (1000 obs)  \\\\\n")
    println("\\hline \\hline")
    param_names = ["\$ \\alpha_1 \$","\$ \\beta_1 \$", "\$ \\Delta_1 \$", 
        "\$ \\alpha_2 \$", "\$ \\beta_2 \$", "\$ \\Delta_2 \$","\$ \\lambda \$" ]
    true_param = ["1", "4", "-3", "2", "3", "-2", ".25"]
    for i=1:7
        print("$(param_names[i]) &")
        print("$(true_param[i]) &")
        print(" $(round(mean(params_200[i,:]), digits=4)) &")
        print(" $(round(std(params_200[i,:]), digits=3)) &")
        print(" $(round(mean(params_1000[i,:]), digits=4)) &")
        print(" $(round(std(params_1000[i,:]), digits=3)) \\\\\n")
        if (i ==3) | (i==6)
            println( "\\hline \\hline")
        else
            println( "\\hline")
        end
        
    end
    println("\\end{tabular}")
    println("\\end{table}")
    println("\\end{center}")
end

print_results(results_200[:,:], results_1000[:,:])

UndefVarError: UndefVarError: results_200 not defined